In [1]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Menyiapkan semua library yang dibutuhkan

In [2]:
# import packages
import csv
import numpy as np
import pandas as pd
import pickle
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers, losses
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt




### Data Wrangling

#### Gathering Data

In [3]:
# import data
dataset1_df = pd.read_csv('/content/sample_data/dataset1.csv')
dataset1_df #show data below

,ingredient_name,rating,functions,link
0,3-O Ethyl Ascorbic Acid,BEST,This potent antioxidant is a highly stable for...,https://www.paulaschoice.com/ingredient-dictio...
1,Acai,BEST,"Pronounced ""ah-sigh-ee"", this small berry has ...",https://www.paulaschoice.com/ingredient-dictio...
2,Acerola Fruit Extract,BEST,This fruit extract is a potent source of antio...,https://www.paulaschoice.com/ingredient-dictio...
3,Acetyl Carnitine HCL,BEST,Functions as an antioxidant.,https://www.paulaschoice.com/ingredient-dictio...
4,Acetyl Dipeptide-1 Cetyl Ester,BEST,A lab-made skin-firming peptide formed from a ...,https://www.paulaschoice.com/ingredient-dictio...
...,...,...,...,...
26082,Zostera Marina Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...,NaN
26083,Zygophyllum Qatarense Leaf/Stem Extract,NOT RATED,Functions: Antioxidant. We have not yet rated ...,NaN
26084,Zygosaccharomyces Microellipsoides Ferment,NOT RATED,Functions: Humectant. We have not yet rated th...,NaN
26085,Zymomonas Ferment Extract,NOT RATED,"Functions: Humectant, Skin Conditioning. We ha...",NaN


In [4]:
# import data
dataset2_df = pd.read_csv('/content/sample_data/dataset2.csv')
dataset2_df #show data below

,ingredient_name,rating,functions,link,benefits,categories,glance,references,all,description,combined_text
0,3-O Ethyl Ascorbic Acid,BEST,This potent antioxidant is a highly stable for...,https://www.paulaschoice.com/ingredient-dictio...,"Anti-Aging, Evens Skin Tone, Dark Spot Fading",Antioxidant,A more stable derivative of pure vitamin C (as...,"Free Radical Biology and Medicine, September ...",INGREDIENT DICTIONARY/SEARCH RESULTS\n3-O Ethy...,"3-O ethyl ascorbic acid is a stable, water- an...","3-O ethyl ascorbic acid is a stable, water- an..."
1,Acai,BEST,"Pronounced ""ah-sigh-ee"", this small berry has ...",https://www.paulaschoice.com/ingredient-dictio...,"Anti-Aging, Soothing","Antioxidant, Plant Extracts",NaN,NaN,INGREDIENT DICTIONARY/SEARCH RESULTS\nAcai\nRa...,Acai (pronounced “ah-sigh-ee”) is a small berr...,Acai (pronounced “ah-sigh-ee”) is a small berr...
2,Acerola Fruit Extract,BEST,This fruit extract is a potent source of antio...,https://www.paulaschoice.com/ingredient-dictio...,Anti-Aging,"Antioxidant, Plant Extracts",NaN,"Journal of AOAC International, September 2018...",INGREDIENT DICTIONARY/SEARCH RESULTS\nAcerola ...,Acerola fruit extract (also known as _Malpighi...,Acerola fruit extract (also known as _Malpighi...
3,Acetyl Carnitine HCL,BEST,Functions as an antioxidant.,https://www.paulaschoice.com/ingredient-dictio...,Hydration,Antioxidant,NaN,Supercharged Hydration;;How Hyaluronic Acid Be...,INGREDIENT DICTIONARY/SEARCH RESULTS\nAcetyl C...,NaN,Functions as an antioxidant.
4,Acetyl Dipeptide-1 Cetyl Ester,BEST,A lab-made skin-firming peptide formed from a ...,https://www.paulaschoice.com/ingredient-dictio...,Anti-Aging,Peptides,Lab-made anti-aging peptide that promotes firm...,"Pharmaceuticals, July 2021, pages 1–22;;Jour...",INGREDIENT DICTIONARY/SEARCH RESULTS\nAcetyl D...,Acetyl dipeptide-1 cetyl ester is a lab-engine...,Acetyl dipeptide-1 cetyl ester is a lab-engine...
...,...,...,...,...,...,...,...,...,...,...,...
26069,Zostera Marina Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Functions: Skin Conditioning. We have not yet...
26070,Zygophyllum Qatarense Leaf/Stem Extract,NOT RATED,Functions: Antioxidant. We have not yet rated ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Functions: Antioxidant. We have not yet rated...
26071,Zygosaccharomyces Microellipsoides Ferment,NOT RATED,Functions: Humectant. We have not yet rated th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Functions: Humectant. We have not yet rated t...
26072,Zymomonas Ferment Extract,NOT RATED,"Functions: Humectant, Skin Conditioning. We ha...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Functions: Humectant, Skin Conditioning. We h..."


### Assesing Data

#### Menilai Tabel dataset1_df

In [5]:
dataset1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26087 entries, 0 to 26086
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ingredient_name  26087 non-null  object
 1   rating           26087 non-null  object
 2   functions        26072 non-null  object
 3   link             2412 non-null   object
dtypes: object(4)
memory usage: 815.3+ KB


In [6]:
dataset1_df.isna().sum()

,0
ingredient_name,0
rating,0
functions,15
link,23675


In [7]:
print("Jumlah Duplikasi: ", dataset1_df.duplicated().sum())

Jumlah Duplikasi:  0


In [8]:
dataset1_df.describe()

,ingredient_name,rating,functions,link
count,26087,26087,26072,2412
unique,26084,6,4055,2412
top,Persea Gratissima (Avocado) Oil,NOT RATED,Functions: Skin Conditioning. We have not yet ...,https://www.paulaschoice.com/ingredient-dictio...
freq,2,23675,5642,1


#### Menilai Tabel dataset2_df

In [9]:
dataset2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26074 entries, 0 to 26073
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ingredient_name  26074 non-null  object
 1   rating           26074 non-null  object
 2   functions        26060 non-null  object
 3   link             2399 non-null   object
 4   benefits         1524 non-null   object
 5   categories       2350 non-null   object
 6   glance           942 non-null    object
 7   references       880 non-null    object
 8   all              2382 non-null   object
 9   description      2370 non-null   object
 10  combined_text    26073 non-null  object
dtypes: object(11)
memory usage: 2.2+ MB


In [10]:
dataset2_df.isna().sum()

,0
ingredient_name,0
rating,0
functions,14
link,23675
benefits,24550
categories,23724
glance,25132
references,25194
all,23692
description,23704


In [11]:
print("Jumlah duplikasi: ", dataset2_df.duplicated().sum())

Jumlah duplikasi:  0


In [12]:
dataset2_df.describe()

,ingredient_name,rating,functions,link,benefits,categories,glance,references,all,description,combined_text
count,26074,26074,26060,2399,1524,2350,942,880,2382,2370,26073
unique,26074,6,4044,2399,90,392,927,367,2382,2350,4245
top,Zymomonas Ferment Filtrate,NOT RATED,Functions: Skin Conditioning. We have not yet ...,https://www.paulaschoice.com/ingredient-dictio...,Hydration,"Antioxidant, Plant Extracts","Plant extract with proven skin-soothing, antio...",Supercharged Hydration;;How Hyaluronic Acid Be...,INGREDIENT DICTIONARY/SEARCH RESULTS\nZingiber...,Used as an emollient and thickening agent in c...,Functions: Skin Conditioning. We have not yet...
freq,1,23675,5642,1,501,255,3,241,1,7,5642


### Cleaning Data

#### Cleaning Tabel dataset1_df

##### Duplicate Data

In [13]:
dataset1_df.duplicated().sum()

0

In [14]:
dataset1_df.drop_duplicates(inplace=True)

In [15]:
print("Jumlah duplikasi: ", dataset1_df.duplicated().sum())

Jumlah duplikasi:  0


##### Missing Value

In [16]:
dataset1_df.isna().sum()

,0
ingredient_name,0
rating,0
functions,15
link,23675


In [17]:
dataset1_df['functions'] = dataset1_df['functions'].fillna("Unknown")

In [18]:
dataset1_df['link'] = dataset1_df['link'].fillna("No Link")

In [19]:
print(dataset1_df.isna().sum())


ingredient_name    0
rating             0
functions          0
link               0
dtype: int64


#### Cleaning Tabel dataset2_df

##### Duplicate Data

In [20]:
dataset2_df.duplicated().sum()

0

In [21]:
dataset2_df.drop_duplicates(inplace=True)

In [22]:
print("Jumlah duplikasi: ", dataset2_df.duplicated().sum())

Jumlah duplikasi:  0


##### Missing Value

In [23]:
dataset2_df.isna().sum()

,0
ingredient_name,0
rating,0
functions,14
link,23675
benefits,24550
categories,23724
glance,25132
references,25194
all,23692
description,23704


In [24]:
dataset2_df['functions'] = dataset2_df['functions'].fillna("Unknown")

In [25]:
print(dataset2_df.isna().sum())

ingredient_name        0
rating                 0
functions              0
link               23675
benefits           24550
categories         23724
glance             25132
references         25194
all                23692
description        23704
combined_text          1
dtype: int64


### Drop Column Link

In [26]:
dataset1_df = dataset1_df.drop(columns='link')

In [27]:
dataset2_df = dataset2_df.drop(columns=['benefits', 'categories', 'glance', 'references', 'all', 'description', 'combined_text', 'link'])

### Exploratory Data Analysis (EDA)

#### Explore Tabel dataset1_df

In [28]:
dataset1_df.sample(5)

,ingredient_name,rating,functions
17681,PEG-45 Bis-(Tocopheryl Succinate),NOT RATED,"Functions: Anti-Sebum, Antioxidant. We have no..."
85,ATP,BEST,"An organic compound from adenosine, which is f..."
20552,Propylene Glycol Diethylhexanoate,NOT RATED,Functions: Emollient. We have not yet rated th...
22031,Saussurea Lappa Root Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...
14829,Meteorite Powder,NOT RATED,"Functions: Hair Conditioning, Skin Conditionin..."


In [29]:
dataset1_df.describe(include="all")

,ingredient_name,rating,functions
count,26087,26087,26087
unique,26084,6,4056
top,Persea Gratissima (Avocado) Oil,NOT RATED,Functions: Skin Conditioning. We have not yet ...
freq,2,23675,5642


In [30]:
dataset1_df.ingredient_name.is_unique

False

In [31]:
dataset1_df.ingredient_name.duplicated

<bound method Series.duplicated of 0                           3-O Ethyl Ascorbic Acid
1                                              Acai
2                             Acerola Fruit Extract
3                              Acetyl Carnitine HCL
4                    Acetyl Dipeptide-1 Cetyl Ester
                            ...                    
26082                        Zostera Marina Extract
26083       Zygophyllum Qatarense Leaf/Stem Extract
26084    Zygosaccharomyces Microellipsoides Ferment
26085                     Zymomonas Ferment Extract
26086                    Zymomonas Ferment Filtrate
Name: ingredient_name, Length: 26087, dtype: object>

In [32]:
#Mengelompokkan ingredients berdasarkan fungsi
aggregated_data = dataset1_df.groupby('ingredient_name').agg({
    'functions': lambda x: ', '.join(set(x.dropna()))
}).reset_index()
print(aggregated_data)


                                         ingredient_name  \
0                                        1, 2-Hexanediol   
1                 1,1-Dimethyl-2-Phenylethyl Isobutyrate   
2                1,1-Dimethyl-3-Phenylpropyl Isobutyrate   
3                                        1,10-Decanediol   
4      1,2,3,4,4A,5,6,7-Octahydro-2,5,5-Trimethyl-2-N...   
...                                                  ...   
26079                          Zymomonas Ferment Extract   
26080                         Zymomonas Ferment Filtrate   
26081                                biosaccharide gum-1   
26082                                        pH Adjuster   
26083                                       polydextrose   

                                               functions  
0      1,2 hexanediol is a synthetic ingredient with ...  
1      Functions: Fragrance. We have not yet rated th...  
2      Functions: Fragrance. We have not yet rated th...  
3      Functions: Solvent. We have not yet 

In [33]:
#Menggabungkan fungsi yang sama untuk setiap ingredients
grouped = dataset1_df.groupby('functions').agg({
    'ingredient_name': 'count'
}).rename(columns={'ingredient_name': 'ingredient_count'})
print(grouped)


                                                    ingredient_count
functions                                                           
1,2 hexanediol is a synthetic ingredient with m...                 1
2,3-butanediol is a synthetic or plant-derived ...                 1
A biomimetic peptide (meaning it is designed to...                 1
A blend of amino acids from wheat protein after...                 1
A blend of fermented filtrates from green tea, ...                 1
...                                                              ...
Yellow 5 lake is synthetic coloring agent appro...                 1
Zinc gluconate is the zinc salt of gluconic aci...                 1
myristamidopropyl PG-dimonium chloride phosphat...                 1
pH adjusting ingredient used to help neutralize...                 1
“Incidentals” refers to any ingredients in cosm...                 1

[4056 rows x 1 columns]


#### Explore Tabel dataset2_df

In [34]:
dataset2_df.sample(5)

,ingredient_name,rating,functions
7807,Copaifera Reticulata Balsam Oil Acetylated,NOT RATED,Functions: Fragrance. We have not yet rated th...
9730,Dodoxynol-6,NOT RATED,"Functions: Cleansing Agent, Emulsifier. We hav..."
19126,Polyacrylate-50,NOT RATED,"Functions: Cleansing Agent, Emulsifier. We hav..."
17127,Passiflora Edulis Seed Oil/Palm Oil Aminopropa...,NOT RATED,Functions: Skin Conditioning. We have not yet ...
13565,Lactobacillus/Clover Flower Ferment Extract,NOT RATED,"Functions: Hair Conditioning, Skin Conditionin..."


In [35]:
dataset2_df.describe(include="all")

,ingredient_name,rating,functions
count,26074,26074,26074
unique,26074,6,4045
top,Zymomonas Ferment Filtrate,NOT RATED,Functions: Skin Conditioning. We have not yet ...
freq,1,23675,5642


In [36]:
dataset2_df.ingredient_name.is_unique

True

In [37]:
dataset2_df.ingredient_name.duplicated

<bound method Series.duplicated of 0                           3-O Ethyl Ascorbic Acid
1                                              Acai
2                             Acerola Fruit Extract
3                              Acetyl Carnitine HCL
4                    Acetyl Dipeptide-1 Cetyl Ester
                            ...                    
26069                        Zostera Marina Extract
26070       Zygophyllum Qatarense Leaf/Stem Extract
26071    Zygosaccharomyces Microellipsoides Ferment
26072                     Zymomonas Ferment Extract
26073                    Zymomonas Ferment Filtrate
Name: ingredient_name, Length: 26074, dtype: object>

In [38]:
#Mengelompokkan ingredients berdasarkan fungsi
aggregated_data = dataset2_df.groupby('ingredient_name').agg({
    'functions': lambda x: ', '.join(set(x.dropna()))
}).reset_index()
print(aggregated_data)

                                         ingredient_name  \
0                                        1, 2-Hexanediol   
1                 1,1-Dimethyl-2-Phenylethyl Isobutyrate   
2                1,1-Dimethyl-3-Phenylpropyl Isobutyrate   
3                                        1,10-Decanediol   
4      1,2,3,4,4A,5,6,7-Octahydro-2,5,5-Trimethyl-2-N...   
...                                                  ...   
26069                          Zymomonas Ferment Extract   
26070                         Zymomonas Ferment Filtrate   
26071                                biosaccharide gum-1   
26072                                        pH Adjuster   
26073                                       polydextrose   

                                               functions  
0      1,2 hexanediol is a synthetic ingredient with ...  
1      Functions: Fragrance. We have not yet rated th...  
2      Functions: Fragrance. We have not yet rated th...  
3      Functions: Solvent. We have not yet 

In [39]:
#Menggabungkan fungsi yang sama untuk setiap ingredients
grouped = dataset2_df.groupby('functions').agg({
    'ingredient_name': 'count'
}).rename(columns={'ingredient_name': 'ingredient_count'})
print(grouped)

                                                    ingredient_count
functions                                                           
1,2 hexanediol is a synthetic ingredient with m...                 1
2,3-butanediol is a synthetic or plant-derived ...                 1
A biomimetic peptide (meaning it is designed to...                 1
A blend of amino acids from wheat protein after...                 1
A blend of fermented filtrates from green tea, ...                 1
...                                                              ...
Yellow 5 lake is synthetic coloring agent appro...                 1
Zinc gluconate is the zinc salt of gluconic aci...                 1
myristamidopropyl PG-dimonium chloride phosphat...                 1
pH adjusting ingredient used to help neutralize...                 1
“Incidentals” refers to any ingredients in cosm...                 1

[4045 rows x 1 columns]


##### Menggabungkan dataset1 dan dataset2

In [40]:
datasetAll_df = pd.concat([dataset1_df, dataset2_df], axis=0, ignore_index=True)
datasetAll_df

,ingredient_name,rating,functions
0,3-O Ethyl Ascorbic Acid,BEST,This potent antioxidant is a highly stable for...
1,Acai,BEST,"Pronounced ""ah-sigh-ee"", this small berry has ..."
2,Acerola Fruit Extract,BEST,This fruit extract is a potent source of antio...
3,Acetyl Carnitine HCL,BEST,Functions as an antioxidant.
4,Acetyl Dipeptide-1 Cetyl Ester,BEST,A lab-made skin-firming peptide formed from a ...
...,...,...,...
52156,Zostera Marina Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...
52157,Zygophyllum Qatarense Leaf/Stem Extract,NOT RATED,Functions: Antioxidant. We have not yet rated ...
52158,Zygosaccharomyces Microellipsoides Ferment,NOT RATED,Functions: Humectant. We have not yet rated th...
52159,Zymomonas Ferment Extract,NOT RATED,"Functions: Humectant, Skin Conditioning. We ha..."


### Export Clean Dataset

In [41]:
# Menyimpan dataset ke file CSV
datasetAll_df.to_csv("/content/sample_data/datasetAll.csv", index=False)


In [42]:
dataset1_df.to_csv("/content/sample_data/dataset1_clean.csv", index=False)
dataset2_df.to_csv("/content/sample_data/dataset2_clean.csv", index=False)

### Pre-processing

In [43]:
# import data
datasetAll_df = pd.read_csv('/content/sample_data/datasetAll.csv')
datasetAll_df #show data below

,ingredient_name,rating,functions
0,3-O Ethyl Ascorbic Acid,BEST,This potent antioxidant is a highly stable for...
1,Acai,BEST,"Pronounced ""ah-sigh-ee"", this small berry has ..."
2,Acerola Fruit Extract,BEST,This fruit extract is a potent source of antio...
3,Acetyl Carnitine HCL,BEST,Functions as an antioxidant.
4,Acetyl Dipeptide-1 Cetyl Ester,BEST,A lab-made skin-firming peptide formed from a ...
...,...,...,...
52156,Zostera Marina Extract,NOT RATED,Functions: Skin Conditioning. We have not yet ...
52157,Zygophyllum Qatarense Leaf/Stem Extract,NOT RATED,Functions: Antioxidant. We have not yet rated ...
52158,Zygosaccharomyces Microellipsoides Ferment,NOT RATED,Functions: Humectant. We have not yet rated th...
52159,Zymomonas Ferment Extract,NOT RATED,"Functions: Humectant, Skin Conditioning. We ha..."


In [44]:
print(datasetAll_df['rating'].unique())

['BEST' 'GOOD' 'AVERAGE' 'BAD' 'WORST' 'NOT RATED']


In [45]:
# Mengonversi nilai non-numerik ke angka
rating_mapping = {
    'BEST': 6,
    'GOOD': 5,
    'AVERAGE': 4,
    'BAD': 3,
    'WORST':2,
    'NOT RATED':1
}

datasetAll_df['rating'] = datasetAll_df['rating'].replace(rating_mapping)

<ipython-input-45-6eb023509551>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datasetAll_df['rating'] = datasetAll_df['rating'].replace(rating_mapping)


In [46]:
# Hapus baris yang tidak sesuai dengan konteks
datasetAll_df = datasetAll_df[datasetAll_df['rating'].apply(lambda x: str(x).isnumeric() or x in rating_mapping.keys())]


In [47]:
datasetAll_df['rating'] = pd.to_numeric(datasetAll_df['rating'], errors='coerce')

In [49]:
datasetAll_df

,ingredient_name,rating,functions
0,3-O Ethyl Ascorbic Acid,6,This potent antioxidant is a highly stable for...
1,Acai,6,"Pronounced ""ah-sigh-ee"", this small berry has ..."
2,Acerola Fruit Extract,6,This fruit extract is a potent source of antio...
3,Acetyl Carnitine HCL,6,Functions as an antioxidant.
4,Acetyl Dipeptide-1 Cetyl Ester,6,A lab-made skin-firming peptide formed from a ...
...,...,...,...
52156,Zostera Marina Extract,1,Functions: Skin Conditioning. We have not yet ...
52157,Zygophyllum Qatarense Leaf/Stem Extract,1,Functions: Antioxidant. We have not yet rated ...
52158,Zygosaccharomyces Microellipsoides Ferment,1,Functions: Humectant. We have not yet rated th...
52159,Zymomonas Ferment Extract,1,"Functions: Humectant, Skin Conditioning. We ha..."


In [53]:
print(datasetAll_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52161 entries, 0 to 52160
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ingredient_name  52161 non-null  object
 1   rating           52161 non-null  int64 
 2   functions        52161 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB
None


### Prepare the dataset for training

In [50]:
vocab_size = 15000
embedding_dim = 64
max_length = 200
trunc_type='post'
padding_type='post'
oov_token = "<OOV>"
training_portion = 0.8

In [51]:
# Tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(datasetAll_df['ingredient_name'])

# Convert ingredient_name to sequences
ingredient_sequences = tokenizer.texts_to_sequences(datasetAll_df['ingredient_name'])

# Padding sequences
ingredient_padded = pad_sequences(ingredient_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [52]:
# Simpan kolom rating dan functions
ratings = datasetAll_df['rating'].tolist()
functions = datasetAll_df['functions'].tolist()


In [56]:
from sklearn.preprocessing import LabelEncoder

# Encode rating
le = LabelEncoder()
encoded_ratings = le.fit_transform(datasetAll_df['rating'])

In [57]:
# Split dataset menjadi training dan testing
X_train, X_test, y_train, y_test, train_functions, test_functions = train_test_split(
    ingredient_padded,
    encoded_ratings,
    datasetAll_df['functions'],
    test_size=0.2,
    random_state=42
)

In [58]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')  # Output sesuai jumlah kategori rating
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [59]:
# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=10,  # Ubah sesuai kebutuhan
    validation_data=(X_test, y_test),
    batch_size=32,
    verbose=1
)

Epoch 1/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9046 - loss: 0.4561 - val_accuracy: 0.9076 - val_loss: 0.4298
Epoch 2/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9071 - loss: 0.4290 - val_accuracy: 0.9076 - val_loss: 0.4238
Epoch 3/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9062 - loss: 0.4336 - val_accuracy: 0.9076 - val_loss: 0.4228
Epoch 4/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9091 - loss: 0.4202 - val_accuracy: 0.9076 - val_loss: 0.4266
Epoch 5/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9090 - loss: 0.4159 - val_accuracy: 0.9076 - val_loss: 0.4157
Epoch 6/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9058 - loss: 0.4243 - val_accuracy: 0.9076 - val_loss: 0.4265
Epoch 7/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9083 - loss: 0.4082 - val_accuracy: 0.9076 - val_loss: 0.4012
Epoch 8/10
1304/1304 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9082 - loss: 0.3945 - 

In [60]:
# Save model and tokenizer
model.save("ingredient_rating_model.h5")

In [61]:
import pickle
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# Load tokenizer dan model untuk prediksi
with open('tokenizer.pkl', 'rb') as f:
    loaded_tokenizer = pickle.load(f)

loaded_model = tf.keras.models.load_model("ingredient_rating_model.h5")

In [63]:
# Prediksi dari teks input
detected_ingredient = "Zostera Marina Extract"  # Contoh input
input_sequence = loaded_tokenizer.texts_to_sequences([detected_ingredient])
input_padded = pad_sequences(input_sequence, maxlen=max_length, padding=padding_type, truncating=trunc_type)

predicted_index = loaded_model.predict(input_padded).argmax(axis=1)[0]
predicted_rating = le.inverse_transform([predicted_index])[0]

# Cari fungsi berdasarkan teks ingredient_name di dataset asli
matching_row = datasetAll_df[datasetAll_df['ingredient_name'].str.lower() == detected_ingredient.lower()]
if not matching_row.empty:
    predicted_function = matching_row['functions'].values[0]
else:
    predicted_function = "Unknown function"

print(f"Detected Ingredient: {detected_ingredient}")
print(f"Predicted Rating: {predicted_rating}")
print(f"Function: {predicted_function}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Detected Ingredient: Zostera Marina Extract
Predicted Rating: 1
Function: Functions: Skin Conditioning. We have not yet rated this ingredient because we have not had a chance to review the research on it.
